In [ ]:
import spacy
import stanza
import textacy
from fastcoref import FCoref
from taxonerd import TaxoNERD
from spacy.matcher import Matcher
from spacy.matcher import DependencyMatcher
# !pip install https://github.com/nleguillarme/taxonerd/releases/download/v1.5.4/en_ner_eco_biobert-1.1.0.tar.gz

In [ ]:
sp_nlp = spacy.load("en_core_web_sm")

In [97]:
def is_split_token(token, prev_token, next_token):
    # if next_token and next_token.pos_ == "VERB":
    # #     return "KEEP_LEFT"
    # if token.pos_ == "VERB" and prev_token.pos_ != "PUNCT" and next_token.pos_ != "PUNCT":
    #     return "SPLIT_PREV"
    # if token.lower_ == "to" and next_token.pos_ in ["VERB", "PROPN"]:
    #     return "SPLIT_LEFT"
    # if token.lower_ == "which":
    #     return True
    if token.pos_ == "CCONJ" and token.head.pos_ not in ["NOUN", "ADP", "ADJ"]:
        return "SPLIT"
    if token.pos_ == "ADP" and next_token and next_token.pos_ in ["NOUN", "PROPN", "DET", "ADJ"]:
        return False
    if token.pos_ in ["SCONJ", "DET", "AUX", "ADP"] and token.lower_ not in ["the", "of", "to", "a", "an"]:
        return "SPLIT"
    return False

def split_sentence(tokens):
    parts = []

    i = 0
    number_tokens = len(tokens)
    for token in tokens:
        # print(f"{token}: {token.pos_}")
        prev_token = None if token.i == 0 else tokens[token.i-1]
        next_token = None if token.i == number_tokens - 1 else tokens[token.i+1]
        split = is_split_token(token, prev_token, next_token)
        if split == "SPLIT":
            parts.append(tokens[i:token.i])
            parts.append(tokens[token.i])
            i = token.i + 1
        elif split == "SPLIT_PREV":
            parts.append(tokens[i:token.i])
            i = token.i
        elif split == "SPLIT_PREV":
            parts.append(tokens[i:token.i-1])
            i = token.i
    if i != number_tokens:
        parts.append(tokens[i:])

    parts = list(filter(lambda p: len(p.text) != 0, parts))
    return parts

texts = [
    "In response to IFNs, Toll-like receptor engagement, or IL-4/IL-13 signaling, macrophages undergo M1 (classical) or M2 (alternative) activation, which represent extremes of a continuum in a universe of activation states.",
    # "The dog, whose tail wagged excitedly, was greeted warmly by his owner.",
    # "Because the children's toys were scattered everywhere, their mother asked them to tidy up.",
    # "While I was at my friend's house, I saw his collection of vintage cars.",
    # "The artist, whose paintings were on display, received a standing ovation.",
    # "If you are looking for the book, please check my sister's desk.",
    # "The man whose car was stolen called the police immediately.",
    # "Although the company's profits had declined, its stock price remained stable.",
    # "Maria's and Jose's new car, a sleek sedan, arrived this morning, much to their delight.",
    # "The Smiths' house, with its sprawling garden, always seemed like a haven of peace.",
    # "After a long and arduous journey, the Martinezes' family, including their children and their pets, finally reached their destination, a cozy cabin nestled in the mountains.",
    "Predation and parasitism each reduced the abundance of the intermediate consumer (Paramecium), and parasitism indirectly reduced the abundance of the basal resource (Serratia).",
    "Grasshoppers exhibited significant diet shifts from grass to herbs (Kruskal-Wallis test, P 0.01, df 3) when they were in the presence of the comparatively sedentary species (the smaller Pisaurina and the larger Hogna) compared to controls without spiders (Fig. 2).",
    "Our results show that phototrophs can indirectly decrease the population density of heterotrophic bacteria by modification of the nature of bacterial interactions with predators.",
    "All predators inflicted significant mortality on the prey at each prey density compared to the predator-free control for that density",
    "Our results show that an increase in sediment organic matter content is associated to a decline in the abundance of Loripes lucinalis (lucinid bivalve) in the Cymodocea nodosa meadows studied, which potentially may weaken the mutualism between the two species.",
    "The abundance of lucinids showed a negative correlation with the organic matter content in vegetated sediments (Fig. 3a), but showed no correlation in bare ones (Fig. 3b).",
    "The MANOVA on the cattle tank experiment showed that the presence of Tramea, nonlethal Anax, and large bullfrog tadpoles all had significant effects on both small tadpole species (Table 1).",
    "Thus the presence of predators, both nonlethal Anax and lethal Tramea, modified the tank environment in a way that facilitated invasion by midges, but only in the absence of large bullfrogs.",
    "We hypothesized that the presence of Anax would decrease foraging activity of small tadpoles, which in turn would decrease predation by Tramea on the small tadpoles",
    "The stranger hit a bystander, the man hit a stranger"
]

for text in texts:
    sp_doc = sp_nlp(text)
    parts = split_sentence(sp_doc)
    print(f"Text: {text}\n\tParts:{parts}\n")

Text: In response to IFNs, Toll-like receptor engagement, or IL-4/IL-13 signaling, macrophages undergo M1 (classical) or M2 (alternative) activation, which represent extremes of a continuum in a universe of activation states.
	Parts:[In response to IFNs, Toll-like receptor engagement, or IL-4/IL-13 signaling, macrophages undergo M1 (classical), or, M2 (alternative) activation, which represent extremes of a continuum in a universe of activation states.]

Text: Predation and parasitism each reduced the abundance of the intermediate consumer (Paramecium), and parasitism indirectly reduced the abundance of the basal resource (Serratia).
	Parts:[Predation and parasitism each reduced the abundance of the intermediate consumer (Paramecium),, and, parasitism indirectly reduced the abundance of the basal resource (Serratia).]

Text: Grasshoppers exhibited significant diet shifts from grass to herbs (Kruskal-Wallis test, P 0.01, df 3) when they were in the presence of the comparatively sedentary

In [ ]:
# Bob ate oranges, apples, and pie, and he also ran